# Slope Analysis

This project is used to segregate cells to the parent groups. This script is designed as a follow-up step after calcium-slope-analysis-bySubGroups.

## Set-Up the Environment

In [ ]:
%load_ext autoreload
import sys
sys.path.append("../src")
import os
import glob
import slopeTools
import plotTools
import statsTools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats
from scipy.optimize import curve_fit

# Pandas DataFrame as a Database


In [ ]:
analyzed = pd.read_csv("AllSniffer.csv")
analyzed

In [ ]:
filterSize = 5
regressionSize = 10
baselineStartIndex = 20
treatmentStartIndex = 30
experimentPeriod = 5/60 #min

length = [len(x) for x in analyzed["Raw dF/F (%)"].values]
maxLength = max(length)
time = np.arange(maxLength)*experimentPeriod 
smoothTimes = statsTools.smoothY(time, filterSize)


# Report response rate by groups and parent cells

In [ ]:
def identifyResponders(df, cell, distance, group, slopeDifferenceThreshold):
    """
    Given a dataframe, group, and distance, return just the cells that change
    more than the given threshold.
    """
    matching = df.loc[df['Group'] == group]
    matching = matching.loc[matching['Distance'] == distance]
    matching = matching.loc[matching['MCN'] == cell]
    totalCellCount = len(matching)
    matching = matching.loc[matching['SlopeDifference (%)'] > slopeDifferenceThreshold] 
    matching = matching.loc[matching['Treatment Slop (%)'] > 5]
    responderCount = len(matching)
    return matching, totalCellCount, responderCount
    
groups = ["KGlu", "CsGlu", "L368"]
distances = [25, 50, 75, 100, 125]
MCNs = []
for MCN in analyzed["MCN"]:
    if MCN in MCNs:
        continue
    else:
        MCNs.append(MCN)

threshold = 12
database = pd.DataFrame(columns = ['Groups','MCN','25µm', '50µm','75µm','100µm', '125µm'])

for group in groups:
    for mcn in MCNs:
        responseByDistance = []
        for distance in distances:
            matching, totalCellCount, responderCount = identifyResponders(analyzed, mcn, distance, group, threshold)
            if totalCellCount == 0:
                responseRate ="NA"
            else:
                responseRate = responderCount*100/totalCellCount
            responseByDistance.append(responseRate)
        row = {'Groups':group, 'MCN': mcn, '25µm':responseByDistance[0], '50µm':responseByDistance[1], '75µm':responseByDistance[2],'100µm':responseByDistance[3],'125µm': responseByDistance[4]}
        database = database.append(row,ignore_index = True)
display(database)
database.to_csv('AllSniffer_byMCN.csv')